In [1]:
import pandas as pd
import numpy as np

In [2]:
games=pd.read_parquet('Datasets/steam_games.parquet')
users_items=pd.read_parquet('Datasets/users_items.parquet')
user_reviews=pd.read_parquet('Datasets/user_reviews.parquet')

In [6]:
users_items.info()


<class 'pandas.core.frame.DataFrame'>
Index: 5153209 entries, 0 to 5170013
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   item_id           int64  
 3   playtime_forever  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 196.6+ MB


In [7]:
user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59305 entries, 0 to 59332
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    59305 non-null  object
 1   item_id    59305 non-null  object
 2   recommend  59305 non-null  bool  
 3   sentiment  59305 non-null  int64 
dtypes: bool(1), int64(1), object(2)
memory usage: 1.9+ MB


In [8]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22528 entries, 88310 to 120443
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        22528 non-null  object 
 1   app_name      22528 non-null  object 
 2   tags          22528 non-null  object 
 3   price         22528 non-null  float64
 4   id            22528 non-null  int64  
 5   developer     22528 non-null  object 
 6   release_year  22528 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 1.4+ MB


def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de retorno:

def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [3]:
df_users_genre=pd.merge(users_items,games[['id','genres','release_year']],left_on='item_id',right_on='id')
df_users_genre.drop(columns=['id','items_count','item_id',],inplace=True)
df_users_genre

,user_id,playtime_forever,genres,release_year
0,76561197970982479,6.0,[Action],2000
1,js41637,0.0,[Action],2000
2,Riot-Punch,0.0,[Action],2000
3,doctr,93.0,[Action],2000
4,corrupted_soul,108.0,[Action],2000
...,...,...,...,...
4039495,peter1023199,6.0,"[Racing, Simulation, Sports]",2016
4039496,76561198071808318,46.0,"[Adventure, Indie]",2016
4039497,76561198081168593,51.0,[Strategy],2016
4039498,76561198107283457,164.0,"[Action, Casual, Indie, Simulation]",2016


In [4]:
df_users_genre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4039500 entries, 0 to 4039499
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   playtime_forever  float64
 2   genres            object 
 3   release_year      object 
dtypes: float64(1), object(3)
memory usage: 123.3+ MB


In [5]:
df_users_genre.isnull().sum()

user_id             0
playtime_forever    0
genres              0
release_year        0
dtype: int64

In [50]:
df_genres_exploded = df_users_genre.explode("genres")
df_genres_exploded


,user_id,release_year,genres,total_playtime
0,--000--,2006,Indie,925.0
1,--000--,2006,Simulation,925.0
2,--000--,2009,Action,5329.0
3,--000--,2010,Action,22.0
4,--000--,2010,Free to Play,22.0
...,...,...,...,...
3339195,zzzmidmiss,2016,Casual,0.0
3339196,zzzmidmiss,2016,Early Access,0.0
3339197,zzzmidmiss,2016,Indie,0.0
3339198,zzzmidmiss,2016,Simulation,0.0


In [9]:
df_genres_exploded.isnull().sum()

user_id             0
playtime_forever    0
genres              0
release_year        0
dtype: int64

In [51]:
#Agrupo por usuario y año y sumo las horas jugadas
df_users_genre=df_genres_exploded.groupby(['user_id','release_year','genres']).agg({'playtime_forever':'sum'}).reset_index()

KeyError: "Column(s) ['playtime_forever'] do not exist"

In [52]:
df_users_genre.rename(columns={'playtime_forever':'total_playtime'},inplace=True)

In [53]:
df_max_playtime_per_genre = df_users_genre.groupby('genres')['total_playtime'].idxmax()
df_users_max_playtime = df_users_genre.loc[df_max_playtime_per_genre]

In [54]:
df_users_max_playtime

,user_id,release_year,genres,total_playtime
1932045,Evilutional,2012,Action,680991.0
2238007,REBAS_AS_F-T,2016,Adventure,788530.0
823370,76561198059330972,2013,Animation &amp; Modeling,65427.0
2060542,KJ72,2014,Audio Production,18940.0
1932047,Evilutional,2012,Casual,636870.0
393475,76561198035718256,2012,Design &amp; Illustration,102554.0
96956,76561197978756659,2013,Early Access,316786.0
823372,76561198059330972,2013,Education,65427.0
1932049,Evilutional,2012,Free to Play,671216.0
2238011,REBAS_AS_F-T,2016,Indie,808265.0


In [55]:
df_users_for_genre = df_users_genre.merge(df_users_max_playtime, on=["user_id", "genres"])


In [57]:
df_users_for_genre[df_users_for_genre['genres']=='Action']

,user_id,release_year_x,genres,total_playtime_x,release_year_y,total_playtime_y
25,Evilutional,2003,Action,0.0,2012,680991.0
26,Evilutional,2006,Action,0.0,2012,680991.0
27,Evilutional,2009,Action,2037.0,2012,680991.0
28,Evilutional,2010,Action,4102.0,2012,680991.0
29,Evilutional,2011,Action,1968.0,2012,680991.0
30,Evilutional,2012,Action,680991.0,2012,680991.0
31,Evilutional,2013,Action,69726.0,2012,680991.0
32,Evilutional,2014,Action,24226.0,2012,680991.0
33,Evilutional,2015,Action,112.0,2012,680991.0
34,Evilutional,2016,Action,1291.0,2012,680991.0


def best_developer_year( año : int ): Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

def developer_reviews_analysis( desarrolladora : str ): Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

Ejemplo de retorno: {'Valve' : [Negative = 182, Positive = 278]}

In [16]:
user_reviews['item_id']=user_reviews['item_id'].astype('int64')
games['id'] = games['id'].astype('int64')

In [24]:
user_reviews['item_id'].isin(games['id']).value_counts()

item_id
True     49057
False    10248
Name: count, dtype: int64

In [29]:
df_dev_reviews=pd.merge(user_reviews,games[['developer','id']],left_on='item_id',right_on='id',how='left')

In [34]:
df_dev_reviews=df_dev_reviews.dropna()


In [41]:
df_dev_reviews=df_dev_reviews.drop(['user_id','recommend','id','item_id'],axis=1)

In [50]:
df_dev_reviews[df_dev_reviews['developer']=='Valve']['sentiment'].value_counts()


sentiment
1    8435
2    1009
0     113
Name: count, dtype: int64

In [118]:
df_dev_counts_reviews = df_dev_reviews.groupby('developer')['sentiment'].value_counts().unstack().reset_index()
df_dev_counts_reviews.columns = ['developer', 'Negative', 'Neutral', 'Positive']

In [119]:
df_dev_counts_reviews[df_dev_counts_reviews['developer']=="Valve"].value_counts()

developer  Negative  Neutral  Positive
Valve      113.0     8435.0   1009.0      1
Name: count, dtype: int64

In [120]:
#Convierto los NaN en 0
df_dev_counts=df_dev_counts_reviews.fillna(0)

In [121]:
#Como podemos ver el valor maximo de las columnas numericas es 8435 por lo que se puede cambiar el tipo de dato a int16
df_dev_counts.max()

developer    高考恋爱委员会,橘子班
Negative           113.0
Neutral           8435.0
Positive          1009.0
dtype: object

In [122]:
df_dev_counts_reviews[['Negative', 'Neutral', 'Positive']] = df_dev_counts[['Negative', 'Neutral', 'Positive']].astype('int16')

In [123]:
df_dev_counts_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2030 entries, 0 to 2029
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   developer  2030 non-null   object
 1   Negative   2030 non-null   int16 
 2   Neutral    2030 non-null   int16 
 3   Positive   2030 non-null   int16 
dtypes: int16(3), object(1)
memory usage: 27.9+ KB


In [124]:
#Se elimina la columna Neutral que no sirve para el endpoint
df_dev_counts_reviews.drop('Neutral',axis=1,inplace=True)

In [125]:
df_dev_counts_reviews.to_parquet(r'Datasets/dev_counts_reviews.parquet')

In [126]:
prueba=pd.read_parquet(r'Datasets/dev_counts_reviews.parquet')
prueba

,developer,Negative,Positive
0,07th Expansion,0,0
1,"10th Art Studio,Adventure Productions",0,0
2,10tons Ltd,0,0
3,11 bit studios,2,1
4,14° East,1,0
...,...,...,...
2025,xXarabongXx,0,0
2026,△○□× (Miwashiba),0,0
2027,"インレ,Inre",0,0
2028,橘子班,0,0
